# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 31 2022 7:33PM,249750,15,ALI014369762,"Alliance Pharma, Inc.",Released
1,Oct 31 2022 7:33PM,249750,15,ALI017110489,"Alliance Pharma, Inc.",Released
2,Oct 31 2022 7:33PM,249750,15,ALI214172400,"Alliance Pharma, Inc.",Released
3,Oct 31 2022 7:33PM,249750,15,ALIC5091310DEN,"Alliance Pharma, Inc.",Released
4,Oct 31 2022 7:33PM,249750,15,ALIC5092797STC,"Alliance Pharma, Inc.",Released
5,Oct 31 2022 7:33PM,249750,15,ALIC5092799PNX,"Alliance Pharma, Inc.",Released
6,Oct 31 2022 7:33PM,249750,15,30011805,"Alliance Pharma, Inc.",Released
7,Oct 31 2022 7:33PM,249750,15,30011804,"Alliance Pharma, Inc.",Released
8,Oct 31 2022 7:33PM,249750,15,30011806,"Alliance Pharma, Inc.",Released
9,Oct 31 2022 7:33PM,249750,15,30011807,"Alliance Pharma, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
16,249746,Released,10
17,249749,Released,6
18,249750,Released,59
19,249751,Released,1
20,249753,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249746,NaN,NaN,10.0
249749,NaN,NaN,6.0
249750,NaN,NaN,59.0
249751,NaN,NaN,1.0
249753,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249680,0.0,0.0,1.0
249685,55.0,0.0,10.0
249693,0.0,0.0,1.0
249712,0.0,0.0,1.0
249713,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
249680,0,0,1
249685,55,0,10
249693,0,0,1
249712,0,0,1
249713,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249680,0,0,1
1,249685,55,0,10
2,249693,0,0,1
3,249712,0,0,1
4,249713,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,249680,,,1
1,249685,55,,10
2,249693,,,1
3,249712,,,1
4,249713,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 31 2022 7:33PM,249750,15,"Alliance Pharma, Inc."
59,Oct 31 2022 6:51PM,249753,21,"NBTY Global, Inc."
60,Oct 31 2022 6:38PM,249751,21,"NBTY Global, Inc."
61,Oct 31 2022 4:42PM,249749,10,ISDIN Corporation
67,Oct 31 2022 4:35PM,249746,10,ISDIN Corporation
77,Oct 31 2022 4:25PM,249744,10,ISDIN Corporation
82,Oct 31 2022 4:00PM,249743,19,"AdvaGen Pharma, Ltd"
83,Oct 31 2022 2:03PM,249730,16,"SHL Pharma, LLC"
84,Oct 31 2022 1:32PM,249725,19,"NAPP Technologies, LLC"
85,Oct 31 2022 1:17PM,249722,20,"Exact-Rx, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 31 2022 7:33PM,249750,15,"Alliance Pharma, Inc.",,,59
1,Oct 31 2022 6:51PM,249753,21,"NBTY Global, Inc.",,,1
2,Oct 31 2022 6:38PM,249751,21,"NBTY Global, Inc.",,,1
3,Oct 31 2022 4:42PM,249749,10,ISDIN Corporation,,,6
4,Oct 31 2022 4:35PM,249746,10,ISDIN Corporation,,,10
5,Oct 31 2022 4:25PM,249744,10,ISDIN Corporation,1,,4
6,Oct 31 2022 4:00PM,249743,19,"AdvaGen Pharma, Ltd",,,1
7,Oct 31 2022 2:03PM,249730,16,"SHL Pharma, LLC",,,1
8,Oct 31 2022 1:32PM,249725,19,"NAPP Technologies, LLC",,,1
9,Oct 31 2022 1:17PM,249722,20,"Exact-Rx, Inc.",12,,11


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 7:33PM,249750,15,"Alliance Pharma, Inc.",59,,
1,Oct 31 2022 6:51PM,249753,21,"NBTY Global, Inc.",1,,
2,Oct 31 2022 6:38PM,249751,21,"NBTY Global, Inc.",1,,
3,Oct 31 2022 4:42PM,249749,10,ISDIN Corporation,6,,
4,Oct 31 2022 4:35PM,249746,10,ISDIN Corporation,10,,
5,Oct 31 2022 4:25PM,249744,10,ISDIN Corporation,4,,1
6,Oct 31 2022 4:00PM,249743,19,"AdvaGen Pharma, Ltd",1,,
7,Oct 31 2022 2:03PM,249730,16,"SHL Pharma, LLC",1,,
8,Oct 31 2022 1:32PM,249725,19,"NAPP Technologies, LLC",1,,
9,Oct 31 2022 1:17PM,249722,20,"Exact-Rx, Inc.",11,,12


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 7:33PM,249750,15,"Alliance Pharma, Inc.",59,,
1,Oct 31 2022 6:51PM,249753,21,"NBTY Global, Inc.",1,,
2,Oct 31 2022 6:38PM,249751,21,"NBTY Global, Inc.",1,,
3,Oct 31 2022 4:42PM,249749,10,ISDIN Corporation,6,,
4,Oct 31 2022 4:35PM,249746,10,ISDIN Corporation,10,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 7:33PM,249750,15,"Alliance Pharma, Inc.",59.0,NaN,NaN
1,Oct 31 2022 6:51PM,249753,21,"NBTY Global, Inc.",1.0,NaN,NaN
2,Oct 31 2022 6:38PM,249751,21,"NBTY Global, Inc.",1.0,NaN,NaN
3,Oct 31 2022 4:42PM,249749,10,ISDIN Corporation,6.0,NaN,NaN
4,Oct 31 2022 4:35PM,249746,10,ISDIN Corporation,10.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 31 2022 7:33PM,249750,15,"Alliance Pharma, Inc.",59.0,0.0,0.0
1,Oct 31 2022 6:51PM,249753,21,"NBTY Global, Inc.",1.0,0.0,0.0
2,Oct 31 2022 6:38PM,249751,21,"NBTY Global, Inc.",1.0,0.0,0.0
3,Oct 31 2022 4:42PM,249749,10,ISDIN Corporation,6.0,0.0,0.0
4,Oct 31 2022 4:35PM,249746,10,ISDIN Corporation,10.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,998959,21.0,0.0,1.0
15,499435,69.0,0.0,55.0
16,749139,2.0,1.0,2.0
19,499468,2.0,0.0,0.0
20,499402,12.0,0.0,12.0
21,998929,4.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,998959,21.0,0.0,1.0
1,15,499435,69.0,0.0,55.0
2,16,749139,2.0,1.0,2.0
3,19,499468,2.0,0.0,0.0
4,20,499402,12.0,0.0,12.0
5,21,998929,4.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,21.0,0.0,1.0
1,15,69.0,0.0,55.0
2,16,2.0,1.0,2.0
3,19,2.0,0.0,0.0
4,20,12.0,0.0,12.0
5,21,4.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,21.0
1,15,Released,69.0
2,16,Released,2.0
3,19,Released,2.0
4,20,Released,12.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,1.0,55.0,2.0,0.0,12.0,0.0
Executing,0.0,0.0,1.0,0.0,0.0,0.0
Released,21.0,69.0,2.0,2.0,12.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,1.0,55.0,2.0,0.0,12.0,0.0
1,Executing,0.0,0.0,1.0,0.0,0.0,0.0
2,Released,21.0,69.0,2.0,2.0,12.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,1.0,55.0,2.0,0.0,12.0,0.0
1,Executing,0.0,0.0,1.0,0.0,0.0,0.0
2,Released,21.0,69.0,2.0,2.0,12.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()